In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from datetime import datetime, timedelta
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import re
import emoji
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
import gensim
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from spellchecker import SpellChecker
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS

In [2]:
import swifter  #apply

In [3]:
# Read CSV file
#df = pd.read_csv(r'C:\Users\Owner\Desktop\Mangimind Data Science Bootcamp\Machine Learning Project 3\tweet_data.csv\tweet_data.csv')
df = pd.read_csv(r'tweet_data.csv')

In [4]:
#Examining general information about the data
#df.info()

#Filling in the NaN values for photoURL and videoURL 
df= df. replace(np.nan,'None',regex=True)

convert_dict = {'tweetID': str,
                'crDate': 'datetime64[ns]',
                'rtUsID': str,
                'usID': str }
df = df.astype(convert_dict)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 785916 entries, 0 to 785915
Data columns (total 18 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   tweetID     785916 non-null  object        
 1   crDate      785916 non-null  datetime64[ns]
 2   edInput     785916 non-null  int64         
 3   editor      785916 non-null  int64         
 4   engages     785916 non-null  int64         
 5   isApproved  785916 non-null  bool          
 6   isEdNeed    785916 non-null  bool          
 7   isRT        785916 non-null  bool          
 8   likes       785916 non-null  int64         
 9   photoUrl    785916 non-null  object        
 10  retweets    785916 non-null  int64         
 11  rtUsID      785916 non-null  object        
 12  text        785916 non-null  object        
 13  topicName   785916 non-null  object        
 14  usFlwrs     785916 non-null  int64         
 15  usID        785916 non-null  object        
 16  us

In [6]:
df.head()

,tweetID,crDate,edInput,editor,engages,isApproved,isEdNeed,isRT,likes,photoUrl,retweets,rtUsID,text,topicName,usFlwrs,usID,usName,videoUrl
0,1070867471245164544,2018-12-07 02:27:55,-1,-1,98,False,True,False,64,https://pbs.twimg.com/media/Dtx8SiIWkAImVsb.jpg,34,-1,The immediate impulse for an alliance of the E...,Business,23464532,5988062,The Economist,None
1,1070868017888837633,2018-12-07 02:30:05,-1,-1,13,False,True,False,10,https://pbs.twimg.com/media/Dtx8yTyW4AEciqP.jpg,3,-1,America's economy is flashing some warning sig...,Business,1732809,16184358,CNN Business,None
2,1070868012864028673,2018-12-07 02:30:04,-1,-1,12,False,True,False,8,None,4,-1,Lyft files for what is expected to be one of t...,Business,2253989,25053299,FORTUNE,None
3,1070867995239555075,2018-12-07 02:30:00,-1,-1,5,False,True,False,4,None,1,-1,"Exporters still waiting to get Rs 6,000 crore ...",Business,1704056,43855487,Business Standard,None
4,1070867995205885952,2018-12-07 02:30:00,-1,-1,5,False,True,False,2,None,3,-1,Ride-hailing firm Lyft races to leave Uber beh...,Business,1997662,15110357,Reuters Business,https://video.twimg.com/amplify_video/10708116...


In [7]:
#Let's have a look at the text 

list(df.iloc[11:19,12]),list(df.iloc[11:19,2])

(['Wall Street recovers slightly from earlier losses spurred by fears of U.S.-China tensions over trade, lower oil prices https://t.co/VPvaB9zruP https://t.co/0YOxbR7cAu',
  'New research suggests you should set harder goals\nhttps://t.co/GwI4wT7Uzr',
  'Judge orders U.S. State Department to disclose possible evidence whether Hillary Clinton used her private email to flout public information requests https://t.co/TTcrPnjQwD',
  '#RajasthanElection2018 LIVE: Voting begins, Satta market bets big on @INCIndia\n\nCatch #live updates here\n\n#RajasthanAssemblyelection2018 #RajasthanElections2018 #RajasthanElections\n\nhttps://t.co/mzTsOMm9an',
  "A room full of Alpine cheese is one of Bloomberg's photos of the week 🧀🧀🧀https://t.co/22LMFT1A6S https://t.co/D6PXWArXyb",
  '#TelanganaElection2018 LIVE: @narendramodi , @AmitShah urge people to vote in large numbers\n\nCatch all the #live updates here \n\n #TelanganaElections #TelanganaAssemblyElections2018 #TelanganaElections2018 #Telangana\n\nh

In [8]:
# Before doing any text processing, please filter the twits
df = df[(df['edInput'] == 1) | (df['edInput'] == 2)]


In [9]:
# See if you have a balanced dataset
df['edInput'].value_counts()

edInput
1    215577
2    106741
Name: count, dtype: int64

In [10]:
# Create a mapping to derive the target variable:
# edInput value of 1 as 1 as these tweets belong to the Business category
# edInput value of 2 as 0 as these tweets doesnt belong to the Business category

df.loc[:, 'Business'] = df['edInput'].map({1: 1, 2: 0})
df['Business'].value_counts(normalize=True)

Business
1    0.668833
0    0.331167
Name: proportion, dtype: float64

In [11]:
# Now, some word frequency analysis
from collections import Counter
def gen_word_freq_plot(tweet_type, words):
    print(f'Number of words in {tweet_type} tweets: {len(words)}')  
    word_freq = Counter(words)
    
    # Top 20 most frequent words in this corpus
    n = 20
    top_n = word_freq.most_common(n)
    words, frequencies = zip(*top_n)  

    # Plot the word frequencies
    fig, ax = plt.subplots(figsize=(6, 3))
    plt.xticks(rotation='vertical')
    plt.xlabel('Words')
    plt.ylabel('Frequency')
    plt.title(f'Frequently used words in the {tweet_type} corpus')
    sns.barplot(x=list(words), y=list(frequencies), ax=ax) 
    plt.show()

In [ ]:
Biz_tweets = df[df['Business'] == 1].copy()
non_Biz_tweets = df[df['Business'] == 0].copy()
Biz_words = Biz_tweets['text'].str.split().sum()
non_Biz_words = non_Biz_tweets['text'].str.split().sum()
gen_word_freq_plot("Business", Biz_words)

In [ ]:
gen_word_freq_plot("non_Biz", non_Biz_words)

In [ ]:
# You can work on exploring the data a little more in depth. 

In [ ]:
# Now, let's see some word cloud
def gen_word_cloud(tweet_type, words):   
    corpus_text = ' '.join(words)

    # Generate word cloud from the corpus
    wordcloud = WordCloud(width=800, 
                          height=400, 
                          background_color='white', 
                         ).generate(corpus_text)

    # Create the plot
    plt.figure(figsize=(7, 7))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(f'Word Cloud in the {tweet_type} corpus')
    plt.axis('off')
    plt.show()

In [ ]:
gen_word_cloud('Business', Biz_words)
gen_word_cloud('non Business', non_Biz_words)

In [ ]:
gen_word_cloud('non Business', non_Biz_words)

In [12]:
# http://t.co is shortened link from Twitter. 
# Twitter automatically shortens all the links within tweets. 
# These links doesnt provide any clue about the category of the tweet. 
# Hence we can remove them.

def remove_twitter_links(text):
    # Define a regex pattern to match the twitter urls http or https
    twitter_url = re.compile(r'https?://t\.co/[^\s]+')

    # Remove the urls in the text by replacing them with ''
    text = twitter_url.sub('', text)

    return text

df['text'] = df['text'].swifter.apply(lambda x: remove_twitter_links(x))
df.head()

Pandas Apply:   0%|          | 0/322318 [00:00<?, ?it/s]

,tweetID,crDate,edInput,editor,engages,isApproved,isEdNeed,isRT,likes,photoUrl,retweets,rtUsID,text,topicName,usFlwrs,usID,usName,videoUrl,Business
721,1070970722598707200,2018-12-07 09:18:12,1,5001,5137,True,True,False,4180,https://pbs.twimg.com/media/DtzaMK_W4AE6Vle.jpg,957,-1,Which one would you choose? 👀,Fashion,37293,1061553474918342656,Gentleman’s style,None,1
1374,1070981773616648193,2018-12-07 10:02:07,1,5003,52650,True,True,True,36349,None,16301,781427301472874497,How it’s made. 😊,Interesting,34897,994950431234080768,Satisfying Slime,https://video.twimg.com/ext_tw_video/107097772...,1
1867,916500037818916866,2017-10-07 03:06:51,2,5003,63872,False,True,True,39698,None,24174,821811651393495040,"Smile, because it confuses people. Smile, beca...",Interesting,966840,482658470,Tips & Tricks Ideas ✌,None,0
2206,1071135340696625152,2018-12-07 20:12:20,1,5001,1714,True,True,False,1261,https://pbs.twimg.com/media/Dt1v6O9XgAIXOrR.jpg,453,-1,𝐶𝑙𝑎𝑢𝑑𝑒 𝑀𝑜𝑛𝑒𝑡,Art,33308,762649146,𝐴𝑟𝑡.,None,1
2263,1071141175606829056,2018-12-07 20:35:31,1,5001,62062,True,True,False,49073,None,12989,-1,Baby Alpacas are so under appreciated.,Animal,1052924,2828212668,Nature is Amazing ☘️,https://video.twimg.com/ext_tw_video/107114109...,1


In [13]:
df.text.head()

,tweetID,crDate,edInput,editor,engages,isApproved,isEdNeed,isRT,likes,photoUrl,retweets,rtUsID,text,topicName,usFlwrs,usID,usName,videoUrl,Business
721,1070970722598707200,2018-12-07 09:18:12,1,5001,5137,True,True,False,4180,https://pbs.twimg.com/media/DtzaMK_W4AE6Vle.jpg,957,-1,Which one would you choose? 👀,Fashion,37293,1061553474918342656,Gentleman’s style,None,1
1374,1070981773616648193,2018-12-07 10:02:07,1,5003,52650,True,True,True,36349,None,16301,781427301472874497,How it’s made. 😊,Interesting,34897,994950431234080768,Satisfying Slime,https://video.twimg.com/ext_tw_video/107097772...,1
1867,916500037818916866,2017-10-07 03:06:51,2,5003,63872,False,True,True,39698,None,24174,821811651393495040,"Smile, because it confuses people. Smile, beca...",Interesting,966840,482658470,Tips & Tricks Ideas ✌,None,0
2206,1071135340696625152,2018-12-07 20:12:20,1,5001,1714,True,True,False,1261,https://pbs.twimg.com/media/Dt1v6O9XgAIXOrR.jpg,453,-1,𝐶𝑙𝑎𝑢𝑑𝑒 𝑀𝑜𝑛𝑒𝑡,Art,33308,762649146,𝐴𝑟𝑡.,None,1
2263,1071141175606829056,2018-12-07 20:35:31,1,5001,62062,True,True,False,49073,None,12989,-1,Baby Alpacas are so under appreciated.,Animal,1052924,2828212668,Nature is Amazing ☘️,https://video.twimg.com/ext_tw_video/107114109...,1


In [14]:
def remove_unnecessary_characters(text):
    # Replace \n with space character
    cleaned_text = re.sub(r'\n+', ' ', text)
   
    # Remove special characters using regex
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)

    # Replace multiple spaces with a single space
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    
    # Remove training and leading spaces
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

In [15]:
# Function to replace emojis and emoticons with textual descriptions
def replace_emojis(text):
    # Replace emojis with their textual descriptions
    text = emoji.demojize(text)
    # Remove the emoji delimiters (colons)
    text = re.sub(r':', '', text)
    return text

# # function for stemming and lemmatization ONLY USE ONE
# def stem_and_lemmatize_tokens(tokens):
#     stemmer = PorterStemmer()
#     lemmatizer = WordNetLemmatizer()
#     return [stemmer.stem(lemmatizer.lemmatize(token)) for token in tokens]

In [16]:
df['text'] = df['text'].swifter.apply(lambda x: remove_unnecessary_characters(x))

Pandas Apply:   0%|          | 0/322318 [00:00<?, ?it/s]

In [42]:
df.text.head(20)

721                            which one would you choose
1374                                         how its made
1867    smile because it confuses people smile because...
2206                                                  NaN
2263                baby alpacas are so under appreciated
2389                         felt cute might delete later
2583                                   sunflower pendants
2619                                           i need one
2660    for some people the idea of compassion entails...
2779                               big hop energy bferber
2829                                  jeopardy is lit now
3683     could you say no to that face bobbyfrenchbulldog
3952                             m o o d whiskeyretriever
4213                       aroma restaurant in rome italy
4414    this is faa sai a rescue elephant she was naug...
4507    me pulling up with all the love and affection ...
4743             a selfrolling burrito kimcheethemaltipoo
5395          

In [18]:
df['text'] = df['text'].swifter.apply(lambda x: remove_unnecessary_characters(x))

Pandas Apply:   0%|          | 0/322318 [00:00<?, ?it/s]

In [19]:
df.head()

,tweetID,crDate,edInput,editor,engages,isApproved,isEdNeed,isRT,likes,photoUrl,retweets,rtUsID,text,topicName,usFlwrs,usID,usName,videoUrl,Business
721,1070970722598707200,2018-12-07 09:18:12,1,5001,5137,True,True,False,4180,https://pbs.twimg.com/media/DtzaMK_W4AE6Vle.jpg,957,-1,Which one would you choose,Fashion,37293,1061553474918342656,Gentleman’s style,None,1
1374,1070981773616648193,2018-12-07 10:02:07,1,5003,52650,True,True,True,36349,None,16301,781427301472874497,How its made,Interesting,34897,994950431234080768,Satisfying Slime,https://video.twimg.com/ext_tw_video/107097772...,1
1867,916500037818916866,2017-10-07 03:06:51,2,5003,63872,False,True,True,39698,None,24174,821811651393495040,Smile because it confuses people Smile because...,Interesting,966840,482658470,Tips & Tricks Ideas ✌,None,0
2206,1071135340696625152,2018-12-07 20:12:20,1,5001,1714,True,True,False,1261,https://pbs.twimg.com/media/Dt1v6O9XgAIXOrR.jpg,453,-1,,Art,33308,762649146,𝐴𝑟𝑡.,None,1
2263,1071141175606829056,2018-12-07 20:35:31,1,5001,62062,True,True,False,49073,None,12989,-1,Baby Alpacas are so under appreciated,Animal,1052924,2828212668,Nature is Amazing ☘️,https://video.twimg.com/ext_tw_video/107114109...,1


In [20]:
def stem_tokens(tokens):
    stemmer = PorterStemmer()
    return [stemmer.stem(token) for token in tokens]

# def lemmatize_tokens(tokens):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(token) for token in tokens]

# def word_lemmatize(string):
#     output = [WordNetLemmatizer().lemmatize(w) for w in string]
#     return output
# df['text_lemm'] = ''
# df['text_lemm'] = df['text'].apply(word_lemmatize)

In [ ]:
# # Convert 'text' column to strings
# df['text'] = df['text'].astype(str)

# # Lowercasing
# df['text'] = df['text'].str.lower()

# # Removing punctuation
# df['text'] = df['text'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

# # Replace emojis and emoticons with textual descriptions
# df['text'] = df['text'].apply(replace_emojis)

# # Tokenization
# df['text'] = df['text'].apply(lambda x: word_tokenize(x))


# Apply stemming or lemmatization
#df['text'] = df['text'].apply(lambda x: stem_tokens(x)) # or lemmatize_tokens(X)
# df['text'] = df['text'].apply(lambda x: lemmatize_tokens(x)) # or stem_tokens(X)

# # Removing stopwords
# stop_words = set(stopwords.words('english'))
# df['text'] = df['text'].apply(lambda x: [word for word in x if word not in stop_words])

# Handling URLs
# df['text'] = df['text'].apply(lambda x: [re.sub(r'http\S+|www\S+', '', word) for word in x])

# # Handling mentions
# df['text'] = df['text'].apply(lambda x: [re.sub(r'@[\w_]+', '', word) for word in x])

# # Handling hashtags
# df['text'] = df['text'].apply(lambda x: [re.sub(r'#\w+', '', word) for word in x])

# Initialize the spell checker
# spell = SpellChecker()

# # Function to correct misspelled words in a text
# def correct_spelling(text):
#     # Tokenize the text into words
#     words = text.split()
    
#     # Find and correct misspelled words
#     corrected_words = []
#     for word in words:
#         # Check if the word is misspelled
#         if not spell.correction(word) == word:
#             # Correct the misspelled word
#             corrected_word = spell.correction(word)
#             corrected_words.append(corrected_word)
#         else:
#             corrected_words.append(word)
    
#     # Join the corrected words back into a sentence
#     corrected_text = ' '.join(corrected_words)
    
#     return corrected_text

# # Apply spell checking to the 'text' column
# df['text'] = df['text'].apply(correct_spelling)
# Print the preprocessed text column
# print(df['text'])

In [ ]:
df.info()

In [ ]:
df['text'].head(10)

In [22]:
import spacy

In [23]:
nlp = spacy.load('en_core_web_sm')

In [26]:
# def lemmatize_tokens(tokens):
#     lemmatizer = WordNetLemmatizer()
#     return [lemmatizer.lemmatize(token) for token in tokens]

# df['text'] = df['text'].apply(lambda x: lemmatize_tokens(x))

def lemmatize_text(text):
    # Process the text
    doc = nlp(text)
    
    lemmatized_text = " ".join([token.lemma_ for token in doc])
    return lemmatized_text
    
# def word_lemmatize(string):
#     output = [WordNetLemmatizer().lemmatize(w) for w in string]
#     return output

# df['text']= df['text'].apply(word_lemmatize)

In [27]:
df.head()

,tweetID,crDate,edInput,editor,engages,isApproved,isEdNeed,isRT,likes,photoUrl,retweets,rtUsID,text,topicName,usFlwrs,usID,usName,videoUrl,Business
721,1070970722598707200,2018-12-07 09:18:12,1,5001,5137,True,True,False,4180,https://pbs.twimg.com/media/DtzaMK_W4AE6Vle.jpg,957,-1,Which one would you choose,Fashion,37293,1061553474918342656,Gentleman’s style,None,1
1374,1070981773616648193,2018-12-07 10:02:07,1,5003,52650,True,True,True,36349,None,16301,781427301472874497,How its made,Interesting,34897,994950431234080768,Satisfying Slime,https://video.twimg.com/ext_tw_video/107097772...,1
1867,916500037818916866,2017-10-07 03:06:51,2,5003,63872,False,True,True,39698,None,24174,821811651393495040,Smile because it confuses people Smile because...,Interesting,966840,482658470,Tips & Tricks Ideas ✌,None,0
2206,1071135340696625152,2018-12-07 20:12:20,1,5001,1714,True,True,False,1261,https://pbs.twimg.com/media/Dt1v6O9XgAIXOrR.jpg,453,-1,,Art,33308,762649146,𝐴𝑟𝑡.,None,1
2263,1071141175606829056,2018-12-07 20:35:31,1,5001,62062,True,True,False,49073,None,12989,-1,Baby Alpacas are so under appreciated,Animal,1052924,2828212668,Nature is Amazing ☘️,https://video.twimg.com/ext_tw_video/107114109...,1


In [28]:
df['lem_text'] = df['text'].swifter.apply(lambda x: lemmatize_text(x))

Pandas Apply:   0%|          | 0/322318 [00:00<?, ?it/s]

In [30]:
df['lem_text'].head()

721                            which one would you choose
1374                                         how its make
1867    smile because it confuse people smile because ...
2206                                                     
2263                 Baby Alpacas be so under appreciated
Name: lem_text, dtype: object

In [34]:
df[['text','lem_text']].head(20)

,text,lem_text
721,Which one would you choose,which one would you choose
1374,How its made,how its make
1867,Smile because it confuses people Smile because...,smile because it confuse people smile because ...
2206,,
2263,Baby Alpacas are so under appreciated,Baby Alpacas be so under appreciated
2389,Felt cute Might delete later,feel cute might delete later
2583,Sunflower pendants,sunflower pendant
2619,I need one,I need one
2660,For some people the idea of compassion entails...,for some people the idea of compassion entail ...
2779,big hop energy bferber,big hop energy bferber


In [43]:
df.text = df.lem_text

In [44]:
df['text'] = df['text'].apply(lambda x: x.lower())

In [45]:
# Replace empty strings with NaN and drop NaN rows
df['text'].replace('', np.nan, inplace=True)

In [46]:
df['text'].head(20)

721                            which one would you choose
1374                                         how its make
1867    smile because it confuse people smile because ...
2206                                                  NaN
2263                 baby alpacas be so under appreciated
2389                         feel cute might delete later
2583                                    sunflower pendant
2619                                           i need one
2660    for some people the idea of compassion entail ...
2779                               big hop energy bferber
2829                                jeopardy be light now
3683     could you say no to that face bobbyfrenchbulldog
3952                             m o o d whiskeyretriever
4213                       aroma restaurant in rome italy
4414    this be faa sai a rescue elephant she be naugh...
4507    i pull up with all the love and affection i ha...
4743               a selfrolle burrito kimcheethemaltipoo
5395          

In [ ]:
df['text'].head(10)

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df['text'].head(50)

What is baseline classifier in machine learning?
A baseline model is essentially a simple model that acts as a reference in a machine learning project. Its main function is to contextualize the results of trained models. Baseline models usually lack complexity and may have little predictive power. Regardless, their inclusion is a necessity for many reasons.

We need a baseline performance and for NLP we just tokenize without doing much.

Removing punctuation
This line uses the apply() method along with a lambda function to remove punctuation from each text in the 'text' column.
The regular expression r'[^\w\s]' matches any non-alphanumeric and non-whitespace characters, and re.sub() replaces them with an empty string.


Tokenization
This line applies the word_tokenize() function from NLTK to tokenize each text in the 'text' column. 
Tokenization splits the text into individual words or tokens.


Removing stopwords
These lines remove stopwords from each text in the 'text' column. First, a set of stopwords for the English language is created using stopwords.words('english'). 
Then, a lambda function is applied using the apply() method, which iterates over each token in the text and keeps only the words that are not in the set of stopwords.

Stemming
These lines perform stemming on each token in the 'text' column using the Porter stemming algorithm.
First, a PorterStemmer object is created. Then, a lambda function is applied using the apply() method, 
which iterates over each token in the text and applies the stemming algorithm to reduce each word to its base or root form.

Lemmatization is the process of reducing words to their base or root form, which is called a lemma. The lemma represents the canonical or dictionary form of a word, from which all inflected forms of the word can be generated.
In English, lemmatization typically involves removing inflections such as plurals, verb conjugations, and adverb or adjective endings to produce the base form of the word. For example, the lemma of the word "running" is "run," and the lemma of the word "better" is "good."

Lemmatization is commonly used in natural language processing (NLP) and text analysis tasks to normalize words and reduce vocabulary size. By reducing words to their lemmas, different forms of the same word are treated as a single token, which can improve the accuracy and efficiency of various NLP algorithms and models.

Question?

- Treat as separate features: If emojis and emoticons carry sentiment or additional meaning that is important for your task, you can treat them as separate features and preserve them in the text. You can encode them uniquely or represent them using special tokens.

The preprocessing steps (replacing emojis and emoticons, lowercasing text, removing punctuation, tokenization, removing stopwords, applying stemming and lemmatization, handling URLs, mentions, and hashtags) are commonly used in text preprocessing for machine learning classifiers. Here's why each step is important:

Replacing emojis and emoticons: Emojis and emoticons are graphical representations of emotions, and they don't carry much meaning in textual analysis. By replacing them with textual descriptions, you can convert them into meaningful words or phrases that can contribute to the understanding of the text.

Lowercasing text: Lowercasing the text helps in normalization by treating different cases of the same word as identical. For example, "apple" and "Apple" will be treated as the same word.

Removing punctuation: Punctuation marks don't usually contribute much to the overall meaning of the text. Removing them simplifies the text and reduces noise in the data.

Tokenization: Tokenization is the process of breaking the text into individual words or tokens. It helps in further analysis and allows the model to understand the context and meaning of each word.

Removing stopwords: Stopwords are common words that occur frequently in a language (e.g., "the", "is", "and"). These words generally don't add much value to the analysis as they are commonly used and don't carry significant meaning. Removing them helps reduce the dimensionality of the data and focuses on more important words.

Applying stemming and lemmatization: Stemming and lemmatization are techniques used to reduce words to their base or root form. This helps in consolidating words with the same meaning and reducing the vocabulary size. For example, "running," "runs," and "ran" can all be stemmed to "run."

Handling URLs, mentions, and hashtags: URLs, mentions of usernames (e.g., "@username"), and hashtags (e.g., "#example") are specific patterns in text that can be treated differently. Handling them involves replacing URLs with a generic token, replacing mentions with a common username, and extracting the meaningful word or phrase from a hashtag.

By applying these preprocessing steps, you can transform the raw text into a format that is more suitable for machine learning classifiers. It helps in reducing noise, capturing important information, and improving the performance of the classifier by focusing on relevant features.Note ONLY USE ONE

Feature vectors are created for machine learning classifiers to represent the input data in a numerical format that can be processed by the algorithms. Here are the key reasons why feature vectors are important for machine learning classifiers:

- Numerical Representation: Machine learning classifiers operate on numerical data. By representing the input data as feature vectors, we can leverage mathematical operations and statistical techniques that are essential for training and making predictions with classifiers.

- Information Extraction: Feature vectors allow us to extract relevant information from the input data. By carefully selecting and designing the features, we can capture the key characteristics or patterns that are indicative of the class labels we want the classifier to learn and predict. These features can include numerical values, text attributes, categorical variables, or any other relevant data representations.

- Model Training and Prediction: Machine learning classifiers learn patterns and relationships between the feature vectors and the corresponding class labels. During the training phase, the classifier analyzes the feature vectors to create a model that can generalize to unseen data. The feature vectors serve as the input to train the classifier, allowing it to learn the underlying patterns and make accurate predictions on new instances.

- Dimensionality Reduction: Feature vectors can help in reducing the dimensionality of the input data. High-dimensional data can be computationally expensive and may suffer from the curse of dimensionality. Feature extraction or selection techniques can be applied to derive a lower-dimensional representation while preserving important information, improving the efficiency and performance of the classifier.

- Standardization and Normalization: Feature vectors can be standardized or normalized to ensure consistent scales across different features. This is particularly important when features have varying units, ranges, or distributions. Standardization allows the classifier to treat all features equally, preventing certain features from dominating the learning process.

- By creating appropriate feature vectors, we enable machine learning classifiers to learn from the data and make accurate predictions or classifications on new instances. The choice and design of feature vectors play a critical role in the performance and effectiveness of machine learning classifiers.

In [ ]:
# # Perform one-hot encoding
# one_hot_encoded = pd.get_dummies(df['topicName'])

# # Concatenate the one-hot encoded columns with the original DataFrame
# df_encoded = pd.concat([df, one_hot_encoded], axis=1)

# # Remove the original 'topicName' column
# df_encoded.drop('topicName', axis=1, inplace=True)

One-hot encoding is commonly used for machine learning classifiers, especially when dealing with categorical variables. Many machine learning algorithms, such as logistic regression, support vector machines, and neural networks, require numerical inputs. One-hot encoding is a technique used to represent categorical variables as binary features, which can be understood and processed by these algorithms.

In one-hot encoding, each category of a categorical variable is represented by a binary feature column. For a variable with n categories, n binary feature columns are created, where each column indicates whether the corresponding category is present or not. The value 1 is assigned to the column representing the category, while all other columns are filled with 0s.

By performing one-hot encoding, categorical variables can be effectively incorporated into the input data, allowing the classifier to learn patterns and make predictions based on these variables. It enables the classifier to understand and utilize the information conveyed by the different categories of the variable.

However, it's important to note that one-hot encoding can increase the dimensionality of the feature space, which may impact the performance of the classifier, especially if the number of categories is large. In such cases, feature selection or dimensionality reduction techniques may be applied to mitigate this issue.

In [41]:
df_encoded['edInput'].value_counts()
# -1 unlabled data
# 1 confirmed by the editor that they are business calss
# 2 they confirmed they are misclassified. Editor does not agree
# 3 editor is confused
# 4 they are business but they were posted previously from another channel. If 4  it will noty be posted ot the user.
# eliminate three and four. Might want to included by relabling 4 as 1.
# stratification means your test splites has the same representation of all the classes. If your original has 30% of label 1 and 60 of label 2 your 

NameError: name 'df_encoded' is not defined

In [ ]:
df_encoded.info()

In [ ]:
filtered_df = df_encoded[df_encoded['edInput'].isin([1,2])]
filtered_df['edInput'] = filtered_df['edInput'].replace(2,0)



In [ ]:
filtered_df['edInput'].value_counts()

In [ ]:
text_data = filtered_df['text']


 There are several machine learning models that are commonly used for text classification. The choice of model depends on various factors such as the size of the dataset, the complexity of the classification task, and the specific requirements of your project. Here are some popular models for text classification:

    Naive Bayes: Naive Bayes is a simple and efficient probabilistic classifier. It works well with text data and is often used as a baseline model for text classification tasks.

    Support Vector Machines (SVM): SVM is a powerful and versatile model for text classification. It can handle high-dimensional data and is known for its ability to find complex decision boundaries.

    Random Forest: Random Forest is an ensemble model that combines multiple decision trees to make predictions. It can handle text data and is robust against overfitting.

    Logistic Regression: Logistic Regression is a simple and interpretable model that works well for binary text classification tasks. It uses a logistic function to model the probability of the input belonging to a certain class.

    Neural Networks: Neural Networks, especially variants like Convolutional Neural Networks (CNNs) and Recurrent Neural Networks (RNNs), have shown promising results in text classification tasks. They can capture complex patterns and dependencies in the text data.

The choice of model ultimately depends on the specific requirements of your project and the characteristics of your data. It's often a good idea to experiment with multiple models and compare their performance using appropriate evaluation metrics to find the best model for your text classification task.

Yes, applying vectorization is typically done on the training dataset in NLP projects. Vectorization is the process of converting text data into numerical representations that can be used as input to machine learning models. This is necessary because most machine learning algorithms require numerical input.

In the context of NLP, vectorization techniques such as bag-of-words (BoW), term frequency-inverse document frequency (TF-IDF), or word embeddings like Word2Vec or GloVe are commonly used. These techniques transform text data into numerical feature vectors that capture the semantic or syntactic information present in the text.

When applying vectorization, it is important to fit the vectorizer (e.g., CountVectorizer or TfidfVectorizer) on the training data and then transform both the training and testing data using the fitted vectorizer. This ensures that the same vocabulary and feature representation are used consistently across the training and testing sets.

By applying both CountVectorizer and TfidfVectorizer, you generate different representations of the text data. CountVectorizer represents the frequency of each word, while TfidfVectorizer represents the importance of each word in the document and the entire corpus. These different representations can be used as inputs for various NLP tasks, such as classification, clustering, or information retrieval.

In [ ]:
X = filtered_df['text'].astype(str)
y = filtered_df['edInput']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Create an instance of CountVectorizer
count_vectorizer = CountVectorizer()

# Apply fit_transform to the training data
X_train_count = count_vectorizer.fit_transform(X_train)

# Apply transform to the testing data
X_test_count = count_vectorizer.transform(X_test)

# Create an instance of TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

# Apply fit_transform to the training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# Apply transform to the testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Create an instance of LogisticRegression with increased max_iter
logistic_regression = LogisticRegression(max_iter=2000)

# Train the logistic regression model using the training data
logistic_regression.fit(X_train_count, y_train)

# Make predictions on the testing data using CountVectorizer features
y_pred_count = logistic_regression.predict(X_test_count)

# Train the logistic regression model using the training data
logistic_regression.fit(X_train_tfidf, y_train)

# Make predictions on the testing data using TF-IDF features
y_pred_tfidf = logistic_regression.predict(X_test_tfidf)

# Evaluate the models
accuracy_count = accuracy_score(y_test, y_pred_count)
accuracy_tfidf = accuracy_score(y_test, y_pred_tfidf)

precision_count = precision_score(y_test, y_pred_count)
precision_tfidf = precision_score(y_test, y_pred_tfidf)

recall_count = recall_score(y_test, y_pred_count)
recall_tfidf = recall_score(y_test, y_pred_tfidf)

f1_count = f1_score(y_test, y_pred_count)
f1_tfidf = f1_score(y_test, y_pred_tfidf)

print("CountVectorizer Accuracy: ", accuracy_count)
print("TF-IDF Accuracy: ", accuracy_tfidf)
print("CountVectorizer Precision: ", precision_count)
print("TF-IDF Precision: ", precision_tfidf)
print("CountVectorizer Recall: ", recall_count)
print("TF-IDF Recall: ", recall_tfidf)
print("CountVectorizer F1-score: ", f1_count)
print("TF-IDF F1-score: ", f1_tfidf)


The conversion of elements in X_train and X_test to strings is necessary because the fit_transform and transform methods of CountVectorizer expect input data in the form of iterable over strings or bytes-like objects.

In the code you provided, the original X_train and X_test are lists, and it's possible that some elements in these lists are not strings. By converting all elements to strings using str(x), we ensure that all elements in X_train and X_test are valid inputs for CountVectorizer.

If your X_train and X_test already consist of strings, you can omit the conversion step and directly use them in the fit_transform and transform methods of CountVectorizer.

In NLP (Natural Language Processing) projects, these evaluation metrics hold particular significance as they help assess the performance of models that deal with text data. Here's their significance in the context of NLP projects:

    Accuracy: In NLP tasks, such as sentiment analysis, text classification, or spam detection, accuracy indicates the overall correctness of the model's predictions. It helps evaluate how well the model can correctly classify or predict the intended labels or categories for the given text data. High accuracy signifies that the model is making correct predictions, which is crucial for reliable results in NLP applications.

   false positive - Precision: Precision is valuable in NLP projects where the focus is on avoiding false positive predictions. For example, in text classification for medical diagnosis or identifying hate speech, precision helps evaluate the model's ability to correctly classify positive instances while minimizing false positives. High precision implies that the model is better at avoiding false positives, ensuring that the predicted positive instances are indeed relevant or meaningful.

    false negative - Recall: Recall is important in NLP projects where capturing all positive instances is crucial, even if it means accepting some false positives. For instance, in information retrieval tasks like search engines or document retrieval systems, high recall indicates that the model can retrieve most of the relevant documents or information, ensuring that fewer relevant instances are missed.

    F1-score: F1-score combines precision and recall into a single metric, making it particularly useful when both avoiding false positives and capturing all positive instances are important. In NLP projects with imbalanced class distributions, where positive or negative instances are sparse, F1-score provides a balanced assessment of the model's performance. It helps strike a balance between precision and recall and is especially relevant when false positives and false negatives have different impacts or costs.

These metrics are essential in evaluating the effectiveness and reliability of NLP models, providing insights into their performance, strengths, and weaknesses. They guide the development and improvement of NLP algorithms, helping researchers and practitioners build robust and accurate models for various NLP applications.

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MaxAbsScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# X = text_data
# y = filtered_df['edInput']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(filtered_df['text'].astype(str), y, test_size=0.2, random_state=42, stratify=y)

# # Create an instance of the TF-IDF vectorizer
# vectorizer = TfidfVectorizer()

# # Apply the vectorizer to the training data
# X_train_tfidf = vectorizer.fit_transform(X_train)

# # Apply the vectorizer to the testing data
# X_test_tfidf = vectorizer.transform(X_test)

# # Scale the TF-IDF data
# scaler = MaxAbsScaler()
# X_train_scaled = scaler.fit_transform(X_train_tfidf)
# X_test_scaled = scaler.transform(X_test_tfidf)

# Define the parameter grid
param_grid = {
    'C': [0.1, 1.0, 10.0],
    'penalty': ['l1', 'l2'],
#    'solver': ['liblinear', 'saga']
}

# Create an instance of logistic regression
logistic_regression = LogisticRegression(max_iter=4000)

# Create an instance of GridSearchCV
grid_search = GridSearchCV(estimator=logistic_regression, param_grid=param_grid, cv=5, n_jobs = -1)

# Perform grid search on the training data
grid_search.fit(X_train_count, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Create a new instance of logistic regression with the best hyperparameters
logistic_regression_best = LogisticRegression(**best_params)

# Train the logistic regression model with the best hyperparameters
logistic_regression_best.fit(X_train_count, y_train)

# Make predictions on the testing data
y_pred = logistic_regression_best.predict(X_train_count)

# Evaluate the model
accuracy = accuracy_score(y_train, y_pred)
precision = precision_score(y_train, y_pred)
recall = recall_score(y_train, y_pred)
f1 = f1_score(y_train, y_pred)

# Print the evaluation metrics
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)

In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC()

# Train the SVM classifier using the training data
svm_classifier.fit(X_train_count, y_train)

# Make predictions on the testing data
y_pred = svm_classifier.predict(X_train_count)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the evaluation metrics
print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1-score: ", f1)

Want to see more classifiers and more nlp pipline process. in count vectorizer, you call the default parameters, there are a few paraemeters you can play with 
GridSearchCV is a class in scikit-learn that provides an automated way to perform hyperparameter tuning using grid search. Grid search is a technique that exhaustively searches the specified hyperparameter values to find the best combination of hyperparameters for a given model.

GridSearchCV takes an estimator (e.g., a classifier or a regressor), a parameter grid (a dictionary specifying the hyperparameter values to search), and a cross-validation strategy as input. It then performs an exhaustive search over all possible combinations of hyperparameters specified in the parameter grid. For each combination, it trains and evaluates the model using cross-validation. The best combination of hyperparameters is determined based on a specified scoring metric.

The main steps involved in using GridSearchCV are as follows:

    Define the parameter grid: Specify the hyperparameters to be tuned and their corresponding values in a dictionary format.

    Create an instance of the estimator: Instantiate the estimator (e.g., a classifier or a regressor) with initial hyperparameter values.

    Create an instance of GridSearchCV: Pass the estimator and parameter grid as arguments to GridSearchCV, along with the desired cross-validation strategy (e.g., number of folds).

    Perform grid search: Call the fit method of GridSearchCV with the training data. This will perform the grid search and find the best combination of hyperparameters.

    Get the best hyperparameters: Access the best hyperparameters using the best_params_ attribute of GridSearchCV.

    Create a new instance of the estimator with the best hyperparameters: Instantiate the estimator with the best hyperparameters obtained from GridSearchCV.

    Train and evaluate the model: Fit the new estimator on the training data and evaluate its performance on the testing data.

Using GridSearchCV helps in automating the process of hyperparameter tuning and finding the best hyperparameter values for your model. It saves you from manually trying out different combinations of hyperparameters and provides a systematic way to optimize your model's performance.

param_grid is a parameter in GridSearchCV that defines the grid of hyperparameters to search. It is a dictionary where the keys represent the hyperparameter names, and the values are lists or arrays of values to be explored during the grid search.

In the context of logistic regression, C, penalty, and solver are commonly used hyperparameters:

    C: In logistic regression, C is the inverse of the regularization strength. It controls the trade-off between fitting the training data well and keeping the model simple to avoid overfitting. Smaller values of C result in stronger regularization, while larger values reduce the regularization effect. Typically, C is a positive float value.

    penalty: The penalty hyperparameter determines the type of regularization to be applied. Regularization helps prevent overfitting by adding a penalty term to the loss function. Common options for penalty in logistic regression are 'l1' (L1 regularization, also known as Lasso) and 'l2' (L2 regularization, also known as Ridge). L1 regularization tends to produce sparse models with some coefficients set to zero, while L2 regularization encourages small weights for all features.

    solver: The solver hyperparameter specifies the algorithm used for optimization during the logistic regression model fitting. Different solvers have different computational characteristics and are suitable for different types of problems. Common choices for solver include 'liblinear', 'lbfgs', 'newton-cg', 'sag', and 'saga'. The 'liblinear' solver is suitable for small-to-medium-sized problems and supports both L1 and L2 penalties.

By providing different values for C, penalty, and solver in the param_grid, you can explore different combinations of hyperparameters to find the best configuration that yields the highest model performance. Grid search will systematically evaluate the model using various combinations of these hyperparameter values and identify the best combination based on the specified scoring metric.



In [ ]:
len(y)

In [ ]:


# # Fix : TOTAL NO. of ITERATIONS REACHED LIMIT.
# logistic_regression = LogisticRegression(max_iter=1000)
# logistic_regression = LogisticRegression(solver='liblinear')

# # Create an instance of LogisticRegression
# logistic_regression = LogisticRegression()

# logistic_regression = LogisticRegression(solver='liblinear')

# # Train the logistic regression model using the training data
# logistic_regression.fit(X_train, y_train.values.ravel())

# # Make predictions on the testing data
# y_pred = logistic_regression.predict(X_test)

# # Evaluate the model
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred)
# recall = recall_score(y_test, y_pred)
# f1 = f1_score(y_test, y_pred)

# print("Accuracy: " + str(accuracy))
# print("Precision: " + str(precision))
# print("Recall: " + str(recall))
# print("F1-score: " + str(f1))

# #next step
# # drop one hot encodded columns, only keep the text column. 
# # our clssifier should only predict bussiness class or not. 

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(bow_features, df_encoded['Business'], test_size=0.2, random_state=42)

# # The purpose of this code is to extract different types of text features, such as Bag-of-Words, TF-IDF representations, 
# # and word embeddings, from the preprocessed text data. 
# # These features can be used as inputs for various machine learning or natural language processing tasks. 
# # The code ensures that feature extraction is performed only when there are valid documents available for analysis.
# # Convert the elements in the 'text' column to strings
# text_data = [' '.join(text) for text in df['text']]

# # Create an instance of CountVectorizer
# bow_vectorizer = CountVectorizer()

# # Apply fit_transform to the text data
# bow_features = bow_vectorizer.fit_transform(text_data)

# # # Print bow_features
# # print(bow_features)

# # Create an instance of TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()

# # Apply fit_transform to the text data
# tfidf_features = tfidf_vectorizer.fit_transform(text_data)

# # Print tfidf_features
# print(tfidf_features)

# # # Word embeddings
# # word2vec_model = gensim.models.Word2Vec(sentences=df['text'], vector_size=100, min_count=1)

# # for text in df['text']:
# #      word_embeddings = word2vec_model.wv[text]
# #      print(word_embeddings)

In [ ]:
# # Convert the elements in the 'text' column to strings
# text_data = [' '.join(text) for text in filtered_df['text']]

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(text_data, filtered_df['edInput'], test_size=0.2, random_state=42, stratify= filtered_df['edInput']  )

# # Create an instance of CountVectorizer
# count_vectorizer = CountVectorizer()

# # Apply fit_transform to the training data
# X_train_count = count_vectorizer.fit_transform(X_train)

# # Apply transform to the testing data
# X_test_count = count_vectorizer.transform(X_test)

# # Create an instance of TfidfVectorizer
# tfidf_vectorizer = TfidfVectorizer()

# # Apply fit_transform to the training data
# X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

# # Apply transform to the testing data
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [ ]:
# X = filtered_df['text'].astype(str)
# y = filtered_df['edInput']

# # Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# # Create an instance of CountVectorizer
# count_vectorizer = CountVectorizer()

# # Apply fit_transform to the training data
# X_train_count = count_vectorizer.fit_transform(X_train)

# # Apply transform to the testing data
# X_test_count = count_vectorizer.transform(X_test)

# # Create an instance of LogisticRegression with increased max_iter
# logistic_regression = LogisticRegression(max_iter=2000)

# # Train the logistic regression model using the training data
# logistic_regression.fit(X_train_count, y_train)

# # Make predictions on the testing data using CountVectorizer features
# y_pred_count = logistic_regression.predict(X_test_count)

# # Evaluate the model
# accuracy_count = accuracy_score(y_test, y_pred_count)
# precision_count = precision_score(y_test, y_pred_count)
# recall_count = recall_score(y_test, y_pred_count)
# f1_count = f1_score(y_test, y_pred_count)

# print("CountVectorizer Accuracy: ", accuracy_count)
# print("CountVectorizer Precision: ", precision_count)
# print("CountVectorizer Recall: ", recall_count)
# print("CountVectorizer F1-score: ", f1_count)